In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Flatten, Softmax,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.layers import Input, Conv2D, Flatten, Softmax
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
import random
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [ ]:
import os

if torch.cuda.is_available():
    device = torch.device('cuda')
    #os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
    #os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
else:
    device = torch.device('cpu')

In [ ]:
tf.__version__

# **Data process**

In [ ]:
#df = open('/kaggle/input/go-competition/29_Training Dataset/Training Dataset/kyu_train.csv').read().splitlines()
#games = [i.split(',',2)[-1] for i in df]
df = open('/kaggle/input/go-competition/29_Training Dataset/Training Dataset/dan_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]

#games = games1 + games2

random.shuffle(games)



In [ ]:
games_sorted = sorted(games, key=lambda x: len(x), reverse=True)

In [ ]:
# 遍历所有游戏数据
for idx, game in enumerate(games):
    num  = 8
    moves_list = game.split(',')
    # 计算当前游戏的长度
    current_length = len(moves_list)
    # 计算需要添加的空移动数量
    if current_length < num:
        padding_length = (num - (current_length % num)) %num
        # 获取第一个动作作为padding_value
        padding_value = 'zore'
        # 使用列表切片添加空移动
        moves_list = [padding_value] * padding_length + moves_list
        # 更新games中的数据
        games[idx] = ','.join(moves_list)

In [ ]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

In [ ]:
def prepare_input(moves):
    x = np.zeros((19,19,3))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            #x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            #x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,2] = 1
    #x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [ ]:
def prepare_input_Zero1(game):
    x_state = np.zeros((19, 19, 2))
    moves_list = game.split(',')
    x_tmp = []
    num = len(moves_list)
    x_return = []
    color = moves_list[0]
    for move in moves_list:
        color = move[0]  
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':  # 黑子
            x_state[row, column, 0] = 1
        if color == 'W':
            x_state[row, column, 1] = 1
        x_tmp.append(x_state)
    for i in range(num-9):
        x = np.zeros((19, 19, 16))
        selected_arrays = x_tmp[i:8+i]
        combined_array = np.concatenate(selected_arrays, axis=-1)
        x[:, :, :16] = combined_array  
        x_return.append(x)
    return x_return
def prepare_label_Zero1(game):
    moves_list = game.split(',')
    num = len(moves_list)
    color = moves_list[0]
    y_tmp = []
    for index, move in enumerate(moves_list):
        if index > 8 :
            column = coordinates[move[2]]
            row = coordinates[move[3]]
            y_tmp.append(column*19+row)

    return y_tmp

        if i % 2 == 0 and color == 'B':
            x[:, :, -1] = 0
            
        elif i%2 == 1 and color == 'B':
            x[:, :, -1] = 1
            
        if i%2 == 0 and color == 'W':
            x[:, :, -1] = 1
            
        elif i%2 == 1 and color == 'W':
            x[:, :, -1] = 0

games_sorted

In [ ]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

In [ ]:
import random


# 使用 random.sample() 函數來從 'games' 中隨機抽取 500 筆數據
random_sample = random.sample(games, 1000)


x = []
y = []
x1 = [] 
y1 = []
for game in random_sample:
        x1 = prepare_input_Zero1(game)
        x.extend(x1)
        y1 = prepare_label_Zero1(game)
        y.extend(y1)
x = np.array(x)
y = np.array(y)
y_one_hot = tf.one_hot(y, depth=19*19)

In [ ]:
x = []
y = []
for game in random_sample:
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        x.append(prepare_input(moves_list[:count]))
        y.append(prepare_label(moves_list[count]))
x = np.array(x)
y = np.array(y)

y_one_hot = tf.one_hot(y, depth=19*19)


In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.05)

In [ ]:
def residual_block(x, num_filters):
    # 保存輸入，用於殘差連接
    shortcut = x
    shortcut = Conv2D(num_filters, kernel_size=3, padding='same')(shortcut)
    # 第一個卷積層
    x = Conv2D(num_filters, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    # 第二個卷積層
    x = Conv2D(num_filters, kernel_size=3, padding='same')(x)
    # 殘差連接
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

In [ ]:
def residual_1(x, num_filters):
    # 保存輸入，用於殘差連接
    shortcut = x
    
    # 第一個卷積層
    x = Conv2D(num_filters, kernel_size=1, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters/2, kernel_size=1, padding='same')(x)
    x = Activation('relu')(x)
    # 第二個卷積層
    x = Conv2D(num_filters, kernel_size=1, padding='same')(x)
    # 殘差連接
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

In [ ]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Softmax
from tensorflow.keras.optimizers import Adam
def create_model():
    inputs = Input(shape=(19, 19, 3))
    x = inputs

    outputs = Conv2D(kernel_size=3, filters=128, padding='same', activation='relu')(x)

    for _ in range(6):  # 您可以根據需要調整殘差塊的數量
        outputs = residual_block(outputs, num_filters=128)
    outputs = residual_1(outputs,128)
    
    for _ in range(6):  # 您可以根據需要調整殘差塊的數量
        outputs = residual_block(outputs, num_filters=128)
    outputs = residual_1(outputs,128)
    
    outputs = Conv2D(kernel_size=1, filters=1, padding='same', activation='relu')(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def create_model():
    inputs = Input(shape=(19, 19, 16))
    outputs = Conv2D(kernel_size=3, filters=256, padding='same', activation='relu')(inputs)
    outputs = Conv2D(kernel_size=3, filters=256, padding='same', activation='relu')(inputs)
    for _ in range(8):  # 您可以根據需要調整殘差塊的數量
        outputs = residual_block(outputs, num_filters=256)
    outputs = residual_1(outputs,256)
    outputs = Conv2D(kernel_size=1, filters=1, padding='same', activation='relu')(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras import mixed_precision
strategy = tf.distribute.MirroredStrategy()
mixed_precision.set_global_policy('mixed_float16')


from keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

initial_learning_rate = 0.000006
min_learning_rate = 0.000001
decay_steps = 5000
decay_rate = 0.9

lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)


with strategy.scope():
    # Load the pre-trained model
    model = load_model('/kaggle/input/testv2/model_dan_testv2_6times_0.5288.h5', compile=False)

    # Define the new optimizer
    new_optimizer =  Adam(learning_rate=lr_schedule)
    new_optimizer = mixed_precision.LossScaleOptimizer(new_optimizer)
    # Compile the model with the new optimizer
    model.compile(optimizer=new_optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.trainable = False
# Display the model summary after recompiling
model.summary()

In [ ]:
for layer in model.layers[-2:]:
    layer.trainable = True

In [ ]:
for layer in model.layers:
    print(f"{layer.name}: 可訓練 - {layer.trainable}")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# 创建 EarlyStopping 回调
early_stopping = EarlyStopping(
    monitor='val_loss',  # 监测验证集损失
    patience=5,           # 当连续 3 个周期验证集损失没有改善时停止训练
    restore_best_weights=True,  # 恢复到验证集损失最低的那个周期的模型权重
)

813/813 [==============================] - 59s 72ms/step - loss: 2.6006 - accuracy: 0.4325 - val_loss: 2.9746 - val_accuracy: 0.38


In [ ]:
accuracy = model.evaluate(x, y_one_hot.numpy())
print(f'Test Accuracy: {accuracy}')

0.4026


In [ ]:
from tensorflow.keras.utils import Sequence
import multiprocessing

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
    # Define your custom data generator class
class MyDataGenerator(Sequence):
    def __init__(self, games, batch_size):
        self.games = games
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.games) / self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        batch_games = self.games[start:end]

        x, y_one_hot = data_pocess(batch_games)
        indices = np.random.permutation(len(x))
        x = x[indices]
        y_one_hot = y_one_hot[indices]
        return x, y_one_hot




In [ ]:
def data_pocess(batch_games):
    with strategy.scope():
        x = []
        y = []
        for game in batch_games:
            moves_list = game.split(',')
            game_x = []
            game_y = []
            for count, move in enumerate(moves_list):
                game_x.append(prepare_input(moves_list[:count]))
                game_y.append(prepare_label(moves_list[count]))
            x.extend(game_x)
            y.extend(game_y)
        x = np.array(x)
        y = np.array(y)
        y_one_hot = tf.one_hot(y, depth=19 * 19)
        return x , y_one_hot.numpy()

In [ ]:
import numpy as np

# 首先將 games 轉換為 NumPy 數組
#games_sorted
# 設定切分比例
split_ratio = 0.95

# 計算切分索引
split_index1 = int(len(games_sorted) * split_ratio)
#split_index2 = int(len(games2) * split_ratio)
#print(split_index1)
#print(len(games))
x_train =  games_sorted #+ games2[:split_index2]

x_val =  games_sorted[40000:40200] #+ games2[split_index2:]

x_train = np.array(x_train)
x_val = np.array(x_val)

random.shuffle(x_train)
random.shuffle(x_val)


In [ ]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in x_train:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

model_dan_testv2_1times_2

In [ ]:
print("model_dan_testv2_7times")

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# 创建 ModelCheckpoint 回调
model_checkpoint = ModelCheckpoint(
    filepath='model_dan_testv2_7times.h5',  # 文件路径，其中 {epoch} 会被替换为周期数
    save_best_only=True,  # 确保保存每个周期的模型，而不仅仅是“最佳”模型
    verbose=1  # 打印保存模型的信息
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Create an instance of the data generator
batch_size = 8
data_generator_train = MyDataGenerator(x_train, batch_size)
data_generator_val = MyDataGenerator(x_val, batch_size)

# 创建 EarlyStopping 回调
early_stopping = EarlyStopping(
    monitor='val_loss',  # 监测验证集损失
    patience=2,           # 当连续 3 个周期验证集损失没有改善时停止训练
    restore_best_weights=True,  # 恢复到验证集损失最低的那个周期的模型权重
)

# Train your model using the data generator
history = model.fit(
    data_generator_train,
    epochs=2,
    validation_data=data_generator_val,
    callbacks=[model_checkpoint, early_stopping],
    workers = 2
)

loss: 1.9665 - accuracy: 0.4883 - val_loss: 1.9501 - val_accuracy: 0.4920

In [ ]:
model.save('./model_dan_testv2_7times.h5')